```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS,KP07, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_convergence_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

## Parameters

In [ ]:
nx = 256
ny = 256

dx = 2000.0
dy = 2000.0

dt = 50
g = 9.81
r = 0.0

# Eddy viscocity parameter
A = 0

# Coriolis parameters: f + beta * y
f = 8.0e-5
beta = 0
y_zero_reference_cell = ny/2 

wind = Common.WindStressParams(type=99)

# Boundary condions
# 1: reflective wall, 
# 2: periodic, 
# 3: open (flow relaxation scheme), 
bcSettings = 3


waterHeight = 50 # m

#Calculate radius from center for plotting
x_center = dx*nx*0.5
y_center = dy*ny*0.5
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


# Simulate the following loop:
# for i in range(T):
#     sim.step(sub_T)
#     sim.download and animate (and write to netCDF)

T = 5
sub_T = 100
make_netCDF = False

bump_posx = 0.5
bump_posy = 0.5
bump_width_factor = 5000 # Pre 2018-01-17: 5000
bump_height = 0.25

reload(Common)


## Forward Backward Linear

In [ ]:
#Clean up old simulator if any:
if 'fbl_sim' in globals():
    fbl_sim.cleanUp()


if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
    validDomain = [None, None, 0, 0]
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] # Both periodic
    validDomain = [1, 1, 0, 0]
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;

eta0 = np.zeros(dataShape, dtype=np.float32);
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);


initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)

eta0 *= 1.1

#Create bump in to lower left of domain for testing
#addBump(eta0, nx-2, ny-2, dx, dy, 0.8, 0.5, 1.0, ghosts)
#addBump(eta0, nx, ny, dx, dy, bump_posx, bump_posy, bump_width_factor, ghosts)
#initializeBalancedVelocityFieldStaggered(eta0, h0, u0, v0, f, beta, g, nx, ny, dx, dy, ghosts)


#Initialize simulator
reload(FBL)
fbl_sim = FBL.FBL(cl_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r, \
              coriolis_beta=beta,\
              y_zero_reference_cell = y_zero_reference_cell, \
              wind_stress=wind, \
              boundary_conditions=boundaryConditions, \
              write_netcdf=make_netCDF)
   
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                                u0[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                                v0[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]])


def animate(i):
    t = fbl_sim.step(sub_T)
    eta1, u1, v1 = fbl_sim.download()

    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                 u1[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                 v1[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]]);

    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    #if (i%20 == 0):
    print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) +  "\tMax eta: " + str(np.max(eta1))
    fig.savefig(imgdir + "/{:010.0f}_fbl.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
eta1, u1, v1 = fbl_sim.download()
eta_diff = (eta1 - eta0)/np.max(eta0)
u_diff = u1 - u0
v_diff = v1 - v0
fig = plt.figure(figsize=(5,5))
plt.imshow(eta_diff[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
plt.colorbar()

fig = plt.figure(figsize=(5,5))
mid_nx = (nx+20)/2
#plt.plot(eta_diff[mid_nx-10:mid_nx+10, y_zero_reference_cell])
plt.plot(eta0[mid_nx-10:mid_nx+10, y_zero_reference_cell])
plt.plot(eta1[mid_nx-10:mid_nx+10, y_zero_reference_cell])



## Centered in time, centered in space

In [ ]:
#Centered in time, centered in space

#Clean up old simulator if any:
if 'ctcs_sim' in globals():
    ctcs_sim.cleanUp()

ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    

initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)


#T = 20
#Initialize simulator
#T = 75
reload(CTCS)
ctcs_sim = CTCS.CTCS(cl_ctx, \
                     h0, eta0, u0, v0, \
                     nx, ny, dx, dy, dt, \
                     g, f, r, A, \
                     coriolis_beta=beta, \
                     y_zero_reference_cell = y_zero_reference_cell, \
                     wind_stress=wind, \
                     write_netcdf=make_netCDF, \
                     boundary_conditions=boundaryConditions )



fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] , 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                                v0[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = ctcs_sim.step(sub_T)
    else:
        t = 0.0
    eta1, u1, v1 = ctcs_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1.0
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                 brighten*v1[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) +  "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if make_netCDF:
    ctcs_sim.cleanUp()

## CDKLM 16

In [ ]:
#Coriolis well balanced reconstruction scheme
if 'cdklm_sim' in globals():
    cdklm_sim.cleanUp()

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
        
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C')
u0   = np.zeros(dataShape, dtype=np.float32, order='C');
v0   = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterHeight

initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)


# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
#addBump(eta0, nx, ny, dx, dy, bump_posx, bump_posy, bump_width_factor, ghosts)
#eta0 = eta0*bump_height
#initializeBalancedVelocityField(eta0, h0, u0, v0, f, 0, g, nx, ny, dx, dy, ghosts)


        
#Initialize simulator
reload(CDKLM16)
cdklm_sim = CDKLM16.CDKLM16(cl_ctx, \
                            eta0, u0, v0, Hi, \
                            nx, ny, dx, dy, dt, \
                            g, f, r, \
                            rk_order=3, \
                            coriolis_beta=beta, \
                            y_zero_reference_cell = y_zero_reference_cell, \
                            #theta=2.0, \
                            wind_stress=wind, \
                            boundary_conditions=boundaryConditions, \
                            write_netcdf=make_netCDF, \
                            double_precision=False, \
                            reportGeostrophicEquilibrium=False, \
                            )


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = cdklm_sim.step(sub_T)
    else:
        t = 0.0
    eta1, u1, v1 = cdklm_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*   u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*   v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if make_netCDF:
     cdklm_sim.cleanUp()

## Kurganov-Petrova 2007

In [ ]:
# Kurganov-Petrova 2007
if 'kp07_sim' in globals():
    kp07_sim.cleanUp()

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
        
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 =   np.zeros(dataShape, dtype=np.float32, order='C');
v0 =   np.zeros(dataShape, dtype=np.float32, order='C');
# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterHeight

initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)



# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
#addBump(eta0, nx, ny, dx, dy, bump_posx, bump_posy, bump_width_factor, ghosts)
#initializeBalancedVelocityField(eta0, h0, u0, v0, f, beta, g, nx, ny, dx, dy, ghosts)

#Initialize simulator
reload(KP07)
kp07_sim = KP07.KP07(cl_ctx, \
                     eta0, Hi, u0, v0, \
                     nx, ny, dx, dy, dt, \
                     g, f, r, \
                     coriolis_beta=beta, \
                     y_zero_reference_cell = y_zero_reference_cell, \
                     wind_stress=wind, \
                     write_netcdf=make_netCDF, \
                     boundary_conditions=boundaryConditions)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = kp07_sim.step(sub_T)
    else:
        t = 0.0
    eta1, u1, v1 = kp07_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if make_netCDF:
    kp07_sim.cleanUp()

## Defining/checking balanced initial conditions
Based on analytic evaluation of the geostrophic balance
$$ hv = \frac{gh}{f} \frac{\partial \eta}{\partial x} $$
and
$$ hu = -\frac{gh}{f} \frac{\partial \eta}{\partial y} $$

In [ ]:

    
makePlot = True    
    
print "Evaluating results for non-staggered grid:"
eta_old = np.zeros(dataShape, dtype=np.float32, order='C')
hu_old   = np.zeros(dataShape, dtype=np.float32, order='C');
hv_old   = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterHeight
# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
addBump(eta_old, nx, ny, dx, dy, bump_posx, bump_posy, bump_width_factor, ghosts)
initializeBalancedVelocityField(eta_old, Hi, hu_old, hv_old, f, 0, g, nx, ny, dx, dy, ghosts)

eta_new = np.zeros(dataShape, dtype=np.float32, order='C')
hu_new   = np.zeros(dataShape, dtype=np.float32, order='C');
hv_new   = np.zeros(dataShape, dtype=np.float32, order='C');
initializeBalancedBumpOverPoint(eta_new, hu_new, hv_new, nx, ny, dx, dy, ghosts, 
                                bump_posx, bump_posy, 1.0, bump_width_factor,
                                f, waterHeight, g)

if makePlot:
    fig = plt.figure(figsize=(4,4))
    plt.imshow(eta_old-eta_new, origin="lower left")
    plt.colorbar()

    fig = plt.figure(figsize=(4,4))
    plt.imshow(hu_old-hu_new, origin="lower left")
    plt.colorbar()

    fig = plt.figure(figsize=(4,4))
    plt.imshow(hv_old-hv_new, origin="lower left")
    plt.colorbar()

print "max(eta_new, eta_old, abs(eta_new-eta_old)): ", np.max(eta_new), np.max(eta_old), np.max(np.abs(eta_new-eta_old))
print "min(eta_new, eta_old, abs(eta_new-eta_old)): ", np.min(eta_new), np.min(eta_old), np.min(np.abs(eta_new-eta_old))
print "max(hu_new, hu_old, abs(hu_new-hu_old)): ", np.max(hu_new), np.max(hu_old), np.max(np.abs(hu_new-hu_old))
print "min(hu_new, hu_old, abs(hu_new-hu_old)): ", np.min(hu_new), np.min(hu_old), np.min(np.abs(hu_new-hu_old))
print "max(hv_new, hv_old, abs(hv_new-hv_old)): ", np.max(hv_new), np.max(hv_old), np.max(np.abs(hv_new-hv_old))
print "min(hv_new, hv_old, abs(hv_new-hv_old)): ", np.min(hv_new), np.min(hv_old), np.min(np.abs(hv_new-hv_old))


print "-----------------------------------------------------------------"
print "Evaluating results for Staggered grid:"
eta_old = np.zeros(dataShape, dtype=np.float32, order='C')
hu_old = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
hv_old = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    

# Bathymetry:
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterHeight
# Add initial conditions (implemented in SWESimulators/BathymetryAndICs.py)
addBump(eta_old, nx, ny, dx, dy, bump_posx, bump_posy, bump_width_factor, ghosts)
initializeBalancedVelocityFieldStaggered(eta_old, Hi, hu_old, hv_old, f, 0, g, nx, ny, dx, dy, ghosts)

eta_new = np.zeros(dataShape, dtype=np.float32, order='C')
hu_new = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
hv_new = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    
initializeBalancedBumpOverPoint(eta_new, hu_new, hv_new, nx, ny, dx, dy, ghosts, 
                                bump_posx, bump_posy, 1.0, bump_width_factor,
                                f, waterHeight, g)

if makePlot:
    fig = plt.figure(figsize=(4,4))
    plt.imshow(eta_old-eta_new, origin="lower left")
    plt.colorbar()

    fig = plt.figure(figsize=(4,4))
    plt.imshow(hu_old-hu_new, origin="lower left")
    plt.colorbar()

    fig = plt.figure(figsize=(4,4))
    plt.imshow(hv_old-hv_new, origin="lower left")
    plt.colorbar()

print "max(eta_new, eta_old, abs(eta_new-eta_old)): ", np.max(eta_new), np.max(eta_old), np.max(np.abs(eta_new-eta_old))
print "min(eta_new, eta_old, abs(eta_new-eta_old)): ", np.min(eta_new), np.min(eta_old), np.min(np.abs(eta_new-eta_old))
print "max(hu_new, hu_old, abs(hu_new-hu_old)): ", np.max(hu_new), np.max(hu_old), np.max(np.abs(hu_new-hu_old))
print "min(hu_new, hu_old, abs(hu_new-hu_old)): ", np.min(hu_new), np.min(hu_old), np.min(np.abs(hu_new-hu_old))
print "max(hv_new, hv_old, abs(hv_new-hv_old)): ", np.max(hv_new), np.max(hv_old), np.max(np.abs(hv_new-hv_old))
print "min(hv_new, hv_old, abs(hv_new-hv_old)): ", np.min(hv_new), np.min(hv_old), np.min(np.abs(hv_new-hv_old))
